In [ ]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 24.3 MB/s eta 0:00:00


In [ ]:
pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 58.0 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    return "\n".join([page.get_text() for page in doc])


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_text(text):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    return splitter.split_text(text)


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

NameError: name 'chunks' is not defined

In [ ]:
import faiss
import numpy as np

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))


In [ ]:
def get_top_k_chunks(query, k=5):
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding), k)
    return [chunks[i] for i in indices[0]]


In [ ]:
from transformers import pipeline

qa = pipeline("text-generation", model="gpt2")  # Replace with more powerful model if needed

def answer_question(query):
    context = "\n".join(get_top_k_chunks(query))
    prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"
    return qa(prompt, max_new_tokens=100)[0]['generated_text']


In [ ]:
import fitz  # PyMuPDF
import numpy as np
import faiss
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter

# ----------------------------
# 1. Extract Text from PDF
# ----------------------------
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    return "\n".join([page.get_text() for page in doc])

# ----------------------------
# 2. Split into Chunks
# ----------------------------
def chunk_text(text):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    return splitter.split_text(text)

# ----------------------------
# 3. Create Embeddings + FAISS Index
# ----------------------------
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

def create_faiss_index(chunks):
    embeddings = embedding_model.encode(chunks)
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(np.array(embeddings))
    return index, chunks

# ----------------------------
# 4. Retrieve Top-k Chunks
# ----------------------------
def get_top_k_chunks(query, index, chunks, k=3):
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), k)
    return [chunks[i] for i in indices[0]]

# ----------------------------
# 5. Generate Answer from Context
# ----------------------------
generator = pipeline("text-generation", model="EleutherAI/gpt-neo-1.3B")


def answer_question(query, index, chunks, max_context_chars=800):
    top_chunks = get_top_k_chunks(query, index, chunks)

    # Join and truncate context safely
    context = "\n".join(top_chunks)
    context = context[:max_context_chars]  # Truncate to avoid token overflow

    prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"

    result = generator(prompt, max_new_tokens=100, do_sample=True, pad_token_id=50256)[0]['generated_text']
    return result.split("Answer:")[-1].strip()


'''def answer_question(query, index, chunks):
    context = "\n".join(get_top_k_chunks(query, index, chunks))
    prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"
    result = generator(prompt, max_new_tokens=100, do_sample=True)[0]['generated_text']
    return result.split("Answer:")[-1].strip()'''

# ----------------------------
# 6. Main Script
# ----------------------------


config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Device set to use cpu


'def answer_question(query, index, chunks):\n    context = "\n".join(get_top_k_chunks(query, index, chunks))\n    prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"\n    result = generator(prompt, max_new_tokens=100, do_sample=True)[0][\'generated_text\']\n    return result.split("Answer:")[-1].strip()'

In [ ]:
if __name__ == "__main__":
    pdf_path = "/content/bbb.pdf"  # Replace with your actual PDF file path
    text = extract_text_from_pdf(pdf_path)
    chunks = chunk_text(text)
    index, stored_chunks = create_faiss_index(chunks)

    print("PDF loaded and indexed. Ask a question!\n")

    while True:
        query = input("Your question (type 'exit' to quit): ")
        if query.lower() in ['exit', 'quit']:
            break
        answer = answer_question(query, index, stored_chunks)
        print("Answer:", answer)


PDF loaded and indexed. Ask a question!

Your question (type 'exit' to quit): vitamin a
Answer: Vitamin A plays an essential role in the body’s vitamin A
signaling and defense system. Research suggests that the amount of vitamin A
needed to prevent an adverse health impact varies from ~250 µg per day for the
best health condition to ~500 µg per day for severe risk conditions.
2
Vitamin A (retinol, retinol palmitate) is a carotenoid that is
composed of at least six different chemical compounds (
Your question (type 'exit' to quit): exit


In [ ]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7ddb9d5e63a0> >